# AI Tutor   

1. Install and Import Gradio
2. Create .env variable and add OpenAI sk
3. Import OpenAI API code snips
4. Load the SS question files as dataframe
    - Output of dataframe 
5. Response functionality should refer to questions in the SS question file
    - The bot should be able to read the file in the 
6. Save responses or csv in txt and use as context for sessions future
7. Prompts engineering should be used to build a very very very good Python Data Science Tutor
    - Preload the tutor wi/Datascienceth Python docu (future feature...use store memory and context in vector database?)mentation
    - Preload the tutor with Data science documentation
    - Tutor should be able to create a game
8. Chatbot buttons
    - Test me : uses database levels 1, 2, 3
    - Track my progress: chatbot checks answer submission and judges if you got answer right or wrong
    - Python flash cards built-in functionality


In [1]:
# Install and Import Gradio, Pandas, Altair, Matplotlib, Os, and OpenAI (put this in "bot_libs.py" file and import it in one line as "import bot_libs" )
import gradio as gr
import pandas as pd
import altair
import matplotlib as plt
import os
import openai
from pathlib import Path #read in txt files for clean bot 
from dotenv import load_dotenv

In [2]:
# Get a list of files and directories in the specified path
os.getcwd()
os.chdir('C:\\Users\\matte\\OneDrive\\Desktop\\Lonely Octopus\\Freelance Project\\Chatbot\\version 1')

## Question Database
Loading questions into script. Decide to use an LLM route to clean the data. Wasn't sure how they felt about scraping their website so went with this route instead. 

In [3]:
# Column Clean Bot function

# Read question database
load_dotenv()

question_df = pd.read_excel('SS questions.xlsx')
openai.api_key = os.getenv("OPENAI_API_KEY") ##This isn't working for some reason
result = load_dotenv()
print(result)
# # Cleaning df (removing rows with #NAME?)
# question_df = question_df[(question_df != '#NAME?').all(axis=1)]
# Use GPT-3.5 to clean the data

def column_clean_bot(prompt, system_content):
    load_dotenv()
    openai.api_key = os.getenv("OPENAI_API_KEY") ##This isn't working for some reason
    messages = [{'role': 'system', 'content': system_content }]
    messages.append({'role': 'user', 'content': prompt})
    

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=1, #play with temp to get more factual responses. max of 2
        max_tokens=25,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    print(response) #print this in the CLI or notebook. This helps with understanding how the response json is structured for further customization
    system_message = response['choices'][0]['message']['content'] #this is parsing the json file that is the response reponse. you take the first item in choices list (a list of dictionaries), go to message key, and then go to content key 
    # messages.append(({'role':'assistant', 'content': system_message})) 
    # messages = [] #this works as a reset to keep the conversation history clear when using in a loop
    return system_message

True


In [ ]:
# Cleaning question column
system_content = Path('question clean prompt.txt').read_text()

for index in range(len(question_df['question'])):
    entry = question_df['question'][index]
    
    if entry == '#NAME?':
        # at the index pick out the question short and python hint, save as variables
        question = question_df['question'][index]  
        hint = question_df['hint'][index]  
        py_solution = question_df['python_solution'][index]
        py_hint = question_df['python_hint'][index]

        # insert those variables into multiline string that inserts variables into the string. save this multiling string as a variable
        prompt = f'''
        QUESTION:
        
        HINT:
        {hint}
        
        PYTHON SOLUTION:
        {py_solution}

        PYTHON HINT:
        {py_hint}

        '''

        # insert multiline string variable into prompt of column_clean_bot
        cleaned_entry = column_clean_bot(prompt, system_content)

        # save clean bot's guess as the new entry at this index
        question_df['question'][index] = cleaned_entry
        

In [ ]:
# # Cleaning hint column
# system_content = Path('hint clean prompt.txt').read_text()

# for index in range(len(question_df['hint'])):
#     entry = question_df['hint'][index]
    
#     if entry == '#NAME?' or entry == '' or entry == None:
#         # at the index pick out the question short and python hint, save as variables
#         question = question_df['question'][index]  
#         hint = question_df['hint'][index]  
#         py_solution = question_df['python_solution'][index]
#         py_hint = question_df['python_hint'][index]

#         # insert those variables into multiline string that inserts variables into the string. save this multiling string as a variable
#         prompt = f'''
#         QUESTION:
#         {question}

#         HINT:
        
#         PYTHON SOLUTION:
#         {py_solution}

#         PYTHON HINT:
#         {py_hint}

#         '''

#         # insert multiline string variable into prompt of column_clean_bot
#         cleaned_entry = column_clean_bot(prompt, system_content)

#         # save clean bot's guess as the new entry at this index
#         question_df['hint'][index] = cleaned_entry

In [ ]:
# # Cleaning python hint column
# system_content = Path('python hint clean prompt.txt').read_text()


# for index in range(len(question_df['python_hint'])):
#     entry = question_df['python_hint'][index]
    
#     if entry == '#NAME?' or entry == '' or entry == None:
#         # at the index pick out the question short and python hint, save as variables
#         question = question_df['question'][index]  
#         hint = question_df['hint'][index]  
#         py_solution = question_df['python_solution'][index]
#         py_hint = question_df['python_hint'][index]

#         # insert those variables into multiline string that inserts variables into the string. save this multiling string as a variable
#         multiline_string = f'''
#         QUESTION:
#         {question}

#         HINT:
#         {hint}
        
#         PYTHON SOLUTION:
#         {py_solution}

#         PYTHON HINT:
        

#         '''

#         # insert multiline string variable into prompt of column_clean_bot
#         cleaned_entry = column_clean_bot(prompt, system_content)

#         # save clean bot's guess as the new entry at this index
#         question_df['python_hint'][index] = cleaned_entry

In [3]:
# save clean df for future use

# question_df.to_csv('question_df.csv', index=False)
question_df = pd.read_csv('question_df.csv')
level_1_df = question_df[question_df['difficulty']== 1]
level_2_df = question_df[question_df['difficulty']== 2]
level_3_df = question_df[question_df['difficulty']== 3]


## Chatbot
This video explained how to use api the best: https://www.youtube.com/watch?v=Si0vFx_dJ5Y

In [10]:
#Tool bot is designed to be the tool-using feature of the chat bot. During the session it will be a second instance of GPT-3.5 that can use tools
#This gives pretty consistent behavior
#save the system_content as a text file and load a variable to make code cleaner
def tool_bot(prompt, history=[]):
    messages =[
        {"role": "system",
         "content": '''
        You are a hyper intelligent robot that understands context, but your default response is the following:
        
        Resonse: xxxxx
        
        The only words you can use are the words below based on the provided condition below. 
        To be clear, If none of the conditions are met you are to respond with """ xxxxx """. You only respond with the following words when conditions are met. 
        Take your time to think this out.
        
        Response Word: GENERAL
        Condition: user asks a general question about data science or python. 
        Examples include: """can you help explain dataframes""", """how do you slice a list"""
        
        Response Word: TEST 
        Condition: user asks to be tested (or any synonym of testing). Only do this when asked explicitly when user asks for a test.

        Response Word: HINT
        Condition: user asks to be given a hint or asks for help (or any synonym of hint). Ignore this logic if the user is asking for an explanation or asking for the solution. Ignore this logic if the user is asking for an explanation or asking for the solution. Ignore this logic if the user is asking for an explanation or asking for the solution.

        Response Word: xxxxx
        Condition: This is the default response when TEST or HINT are not appropriate

        Response Word: LEVEL 1
        Condition: If the user resonds with the numerical level of question they want (in this case """1"""), please issue command """LEVEL 1""". Please read the user input very carefully. If user is returning numbers in context of something calculation or code, please ignore.
        Condition: If the user resonds with the numerical level of question they want (in this case """1"""), please issue command """LEVEL 1""". Please read the user input very carefully. If user is returning numbers in context of something calculation or code, please ignore.
        Condition: If the user resonds with the numerical level of question they want (in this case """1"""), please issue command """LEVEL 1""". Please read the user input very carefully. If user is returning numbers in context of something calculation or code, please ignore.
        Example user inputs include: """"level 1"""", """ 1 """", "lvl 1", "one", "I'd like level 1" etc

        Response Word: LEVEL 2
        Condition: The user responds with the numerical level of question (in this case  """2""" ). Please read the user input very carefully. If user is returning numbers in context of something calculation or code, please ignore.
        Examples include: """"level 2"""", """ 2 """", "lvl 2", "two", "I'd like level 2" etc

        Response Word: LEVEL 3
        Condition: The user responds with a numerical level of question ( in this case """3"""). Please read the user input very carefully. If user is returning numbers in context of something calculation or code, please ignore.
        Examples include: """"level 3"""", """ 3 """", "lvl 3", "three", "I'd like level 3" etc

        Response Word: SOLUTION
        Condition: The user responds with a code snipet or declares that they are submitting a solution (or any synonym of solution)
        Examples include: """This is my answer...""", """Answer:....""", """Here is my solution...""" etc

     
        
        '''}]
       
    messages.append({'role': 'user', 'content': f'Here is my question: {prompt}'})#using f string to avoid prompt injections
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0, #play with temp to get more factual responses
        max_tokens=10,#further limit output
        top_p=0,
        frequency_penalty=0,
        presence_penalty=0
    )


    # saving the message we want to show to the user
    assistant_content = response['choices'][0]['message']['content']

    # Visual check for testing purposes
    print(response) 

    
    #saving as variable to manipulate later
    format_str = f'''
    User: {prompt}
    
    Response: {assistant_content}
    
    '''

    # Saving content to a text file
    # Open a file in write mode, 'a' argument serves as appending to
    with open('saved_chat.txt', 'a') as file:
        file.write(format_str)

    #add token counter and a way to handle hitting token limit. 
    # Prob summarizing previous assistant/student info into 2-4 sentences, 3000 tokens is safe place to start summary
    token_count = response['usage']['total_tokens']
    if token_count >= 3000:
        # Reload message history text for summarization. Better formatted than parsing the response JSON
        # content_txt = Path('saved_chat.txt').read_text() 
        # summary_str[0] = summarize_bot_saved(content_txt)
        messages = [messages[0]] # basically erase the memory except the system message and kept it as a list
        # print(f'This is the `messages` memory: {summary_str}')
    
    # Append messages memory  
    messages.append({'role':'assistant', 'content': assistant_content})
    #if error is token limit, ask the user to submit a shorter message by approximately x words (limit of the system)
    return assistant_content
   
# #testing tool_bot
# prompt = input()
# tool_bot(prompt=prompt) 

In [11]:
#The main chotbot. it will have a nested openai api call in the form of the tool_bot function
# Which will trigger some either grabbing data from questions database
# 
from dotenv import load_dotenv
#may need to repeat commands to ensure that it doesn't deviate from the behavior
load_dotenv()
#global list of messages. 
ai_tutor_sys_content = Path('ai_tutor_sys_content.txt').read_text()
messages =[
        {"role": "system","content": ai_tutor_sys_content }]
#setting global variables, such that when I run my ai_chat function, hint is stored.
#the only way to store memory outside of a function based on action happening within a function without using 
#return statement is to use mutable objects. initiate an empty list and then add or replace items in that list
# we store the 0 index with an placeholder '' string so that we don't get index error for index being out of range. 
hint = ['']
py_hint = ['']
question = ['']
py_solution = ['']
def ai_chat(prompt, history):
    global messages
    #should I start with an assistant message to help the user get started?]#list of messages that get passed in to start convo, will also save messages here

    openai.api_key = os.getenv("OPENAI_API_KEY") ##This isn't working for some reason 
    
    messages.append({'role': 'user', 'content': f'{prompt}'}) #f string to avoid prompt injection errors from user
  
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=1, #play with temp to get more factual responses
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )


    tool_response = tool_bot(prompt) #this is another instance of OpenAI (basically another agent) to issue commands
    #initating empty variables for use later:
   
    if tool_response == 'TEST':
        print('testing TEST')
        assistant_content = ('What level would you like? Please choose level 1, 2, or 3.')
        # ask for level between 1 - 3
        # load filtered data frame of question of that level
        # make this the assistant message
    elif tool_response == 'LEVEL 1':
        random_row = level_1_df.sample(n=1)
        question[0] = random_row['question'].to_list()
        hint[0] = random_row['hint'].to_list()
        py_hint[0] = random_row['python_hint'].to_list()
        py_solution[0] = random_row['python_solution'].to_list()
        assistant_content = f'''Here is your question:
        
        {question[0]}
        
        Please answer to the best of your ability in Python. If you need a hint don't hesistate to ask for one. 
        You can also ask me questions about a certain topic and I will do my best to help you. 
        
        '''
        # return system_message #perhaps storing these as attributes of an object is better?
    elif tool_response == 'HINT': # output both code and non-code hints
        print('testing HINT')
        assistant_content = f'''Here is a hint: 

        {hint[0]}

        If you have anymore questions about the solution. Otherwise please submit a solution.
        '''
    elif tool_response == 'SOLUTION': # output both code and non-code hints
        print('testing SOLUTION')
        assistant_content = f'''Here is the solution: 

        {py_solution[0]}

        If you have anymore questions about the solution.
        '''
    # Add a nested "explain bot" functionality? for MVP perhaps best to stick with relying GPT prompt engineering    
    else:
        assistant_content = response['choices'][0]['message']['content']
    # hint = '' #this is parsing the json file that is the response reponse. you take the first item in choices list (a list of dictionaries), go to message key, and then go to content key 
    
    #print this in the CLI or notebook. This helps with understanding how the response json is structured for further customization
    print(f'''
    
    MESSAGE: {messages}
    
    RESPONSE:{response}
    
    ''') 
    # assistant_content = response['choices'][0]['message']['content']

    # Visual check for testing purposes
    print(response) 

    
    #saving as variable to manipulate later
    format_str = f'''
    User: {prompt}
    
    Response: {assistant_content}
    
    '''

    # Saving content to a text file
    # Open a file in write mode, 'a' argument serves as appending to
    with open('saved_chat.txt', 'a') as file:
        file.write(format_str)

    #add token counter and a way to handle hitting token limit. 
    # Prob summarizing previous assistant/student info into 2-4 sentences, 3000 tokens is safe place to start summary
    token_count = response['usage']['total_tokens']
    if token_count >= 3000:
        # Reload message history text for summarization. Better formatted than parsing the response JSON
        # content_txt = Path('saved_chat.txt').read_text() 
        # summary_str[0] = summarize_bot_saved(content_txt)
        messages = [messages[0]] # basically erase the memory except the system message and kept it as a list
        # print(f'This is the `messages` memory: {summary_str}')
    
    #if keyword command in system_message, 
    #first assistant message should be what level (and language) do you want?
    #call a function that outputs a random row in questions dataframe based on level (and language)
    
    messages.append(({'role':'assistant', 'content': assistant_content}))
    # messages=[] ##to clear memory and reset tokens maybe need to do this after adding save function for chat history and if token count is getting close to 4000. 
    

    return assistant_content

demo = gr.ChatInterface(
    fn=ai_chat, 
    submit_btn = 'Submit', 
    retry_btn = 'Retry',
    clear_btn = 'Clear',
    title = 'AI Tutor V1',
    #add a save convo button that can create html file that can then be printed like a pdf later
    )

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


{
  "id": "chatcmpl-7mfWpY1cRw8G2ajp8ZGtydRCdBGFt",
  "object": "chat.completion",
  "created": 1691833735,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "xxxxx"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 737,
    "completion_tokens": 2,
    "total_tokens": 739
  }
}

    
    MESSAGE: [{'role': 'system', 'content': "You are an data science tutor with 20+ years of experience but you also follow instructions very well. You are speaking to a student who has 0-5 years of experience in the data science field. Here are some instructions for you:\n\n1. In this set of instructions, everything in between ``` and ``` characters are the exact format you should ALWAYS ALWAYS ALWAYS use. Text between '<' and '>' characters are descriptions of what type of response to insert to the text preceding it.\n\n2. You will be given access to a question database which has Pyt

## A hint I found during my chat gpt session "AI Tutor Chat"
You can assign a variable based on a conditional statement inside a function. However, any variable defined inside a function in Python is local to that function, and it can't be accessed outside of the function unless it's returned or made global. 

Here's how you can do it with a `return` statement:

```python
def check_condition(condition):
    if condition:
        result = "The condition was True."
    else:
        result = "The condition was False."
    return result

# usage
my_result = check_condition(True)
print(my_result)  # prints: The condition was True.
```

If you don't want to use a `return` statement, you have two options:

1. **Use a global variable:** You can define a variable outside the function and use the `global` keyword inside the function to tell Python that you want to use the global variable.

```python
result = None

def check_condition(condition):
    global result
    if condition:
        result = "The condition was True."
    else:
        result = "The condition was False."

# usage
check_condition(True)
print(result)  # prints: The condition was True.
```

2. **Use mutable data types:** If you use a mutable data type like a list or a dictionary, changes made inside the function will be reflected outside of it, because the function receives a reference to the original data, not a copy of it.

```python
def check_condition(condition, result):
    if condition:
        result.append("The condition was True.")
    else:
        result.append("The condition was False.")

# usage
my_result = []
check_condition(True, my_result)
print(my_result)  # prints: ['The condition was True.']
```

Each of these methods has its pros and cons. Using global variables can make code harder to understand and debug, especially in larger programs, because it's not immediately clear where the variable is being changed. Using mutable data types can make your code more complex, especially if the function needs to change multiple values. In general, using `return` is the simplest and most straightforward way to get data out of a function.

In [ ]:
''' simplify database, only save the question_short and the solution. Bot then parses html of current page user is on in 
order to look up what question and the solution are as context, and dynamically generates hints and walkthrough. 
This can be saved to a csv for future reference as well. 

can also save all conversations that can be analyzed for future referece by the team at statascratch. Can use an LLM to
then categorize types of questions asked and perform analytics.'''

In [ ]:
# Add functionality that tracks tokens and keeps system message, but deletes or summarizes conversation into a few lines, and feeds resets token count back to where it was at beginning of convo.
#FAQ database:
#Save responses, create an openai instance that can read each entry column one at atime (to avoid token limit)
# and categorize commonly asked responses...and save that as a csv for future analysis

#Future functionality:
#Human like responsiveness. randomized delayed timer
#Voice optionality. 
